# C++ Implementation of Inductive VennABERS Predictor (IVAP)

This is a relatively pedestrian translation in C++ of the Python implementation of the IVAP.

This notebook runs under Cern's ROOT environment. In JupyROOT the Python kernel can also JIT compile C++.
Also the notebook expects that `libfmt-dev` be installed on the system.


In [1]:
import ROOT

Welcome to JupyROOT 6.30/04


In [2]:
ROOT.gSystem.AddIncludePath(" -I/usr/local/include/")

In [3]:
%%cpp
#include <fmt/core.h>
#include <fmt/ranges.h>

In [4]:
ROOT.gSystem.Load("libfmt.so")

0

In [5]:
%%cpp
fmt::print("Hello!");

Hello!

In [6]:
from VennABERS import *

In [7]:
%%cpp
#include <vector>
#include <map>
#include <vector>
#include <algorithm>
#include <cmath>

In [8]:
%%cpp -d
using T = double;

std::tuple<std::vector<T>, std::vector<int>, std::vector<int>> unique_with_counts_and_inverse(const std::vector<T>& x) {

  // Find unique elements and their counts
  std::map<T, int> counts;

  for (const T& value : x) {
    if (auto search=counts.find(value); search != counts.end()) {  
      counts[value]++;
    } else {
      counts[value] = 1;
    }
  }

  std::vector<T> unique;
  std::vector<int> counts_vec;
  unique.reserve(counts.size());
  counts_vec.reserve(counts.size());

  std::vector<int> inverse(x.size());

  for (const auto& [val,count] : counts) {
    unique.push_back(val);
    counts_vec.push_back(count);
  }

  // Find inverse indices
  for (int i = 0; i < x.size(); ++i) {
    inverse[i] = std::lower_bound(unique.begin(), unique.end(), x[i]) - unique.begin();
  }

  return std::make_tuple(unique, counts_vec, inverse);
}


In [9]:
%%cpp
std::vector<double> tv{1.0,3.5,1.0,2.0,4.0,4.0};

In [10]:
%%cpp
std::vector<T> u;
std::vector<int> c;
std::vector<int> i;
std::tie(u,c,i) = unique_with_counts_and_inverse(tv)

(std::tuple &) { { 1.0000000, 2.0000000, 3.5000000, 4.0000000 }, { 2, 1, 1, 2 }, { 0, 2, 0, 1, 3, 3 } }


In [11]:
%%cpp -d
using Point = std::pair<double, double>;

Point top(const std::vector<Point>& S) {
    return S[S.size()-1];
}

Point nextToTop(const std::vector<Point>& S) {
    return S[S.size()-2];
}


bool nonLeftTurn(const Point& a, const Point& b, const Point& c) {
  double d1x = b.first - a.first, d1y = b.second - a.second;
  double d2x = c.first - b.first, d2y = c.second - b.second;
  return (d1x * d2y - d1y * d2x) <= 0.0;
}

bool nonRightTurn(const Point& a, const Point& b, const Point& c) {
  return nonLeftTurn(c, b, a); // Equivalent due to reversed order
}

double slope(const Point& a, const Point& b) {
  if (a.first == b.first) {
    return std::numeric_limits<double>::infinity(); // Handle vertical lines
  }
  return (b.second - a.second) / (b.first - a.first);
}

bool notBelow(const Point& t, const Point& p1, const Point& p2) {
  double m = slope(p1, p2);
  double b = p2.second - m * p2.first;
  return t.second >= m * t.first + b;
}

# Algorithm1 and Algorithm2

In [12]:
%%cpp -d
// We shift all the indices for P and make it a vector
std::vector<Point> algorithm1(const std::vector<Point>& P, int kPrime) {
  std::vector<Point> S;

  // Initialize stack with first two points
  S.push_back(P[-1+1]);
  S.push_back(P[0+1]);

  // Iterate from 1 to kPrime (inclusive)
  for (int i = 1; i <= kPrime; ++i) {
    while (S.size() > 1 && nonLeftTurn(nextToTop(S), top(S), P[i+1])) {
      S.pop_back();
    }
    S.push_back(P[i+1]);
  }

  return S;
}

std::vector<double> algorithm2(std::vector<Point>& P, const std::vector<Point>& S, int kPrime) {
  std::vector<Point> Sprime(S.rbegin(), S.rend()); // Reverse stack
  std::vector<double> F1(kPrime + 1);

  F1[0] = 0.0;
  // Iterate from 1 to kPrime (inclusive)
  for (int i = 1+1; i <= kPrime+1; ++i) {    // i shifted by 1
    F1[i - 1] = slope(top(Sprime), nextToTop(Sprime));
    P[i - 1].first  = P[i - 2].first  + P[i].first  - P[i - 1].first;
    P[i - 1].second = P[i - 2].second + P[i].second - P[i - 1].second;

    if (notBelow(P[i - 1], top(Sprime), nextToTop(Sprime))) {
      continue;
    }

    Sprime.pop_back();
    while (Sprime.size() > 1 && nonLeftTurn(P[i - 1], top(Sprime), nextToTop(Sprime))) {
      Sprime.pop_back();
    }
    Sprime.push_back(P[i - 1]);
  }
  return F1;
}

# Algorithm3 and Algorithm4

In [13]:
%%cpp -d
std::vector<Point> algorithm3(const std::vector<Point>& P, int kPrime) {
  std::vector<Point> S;
  S.push_back(P[kPrime + 1]); // Push P[kPrime+1] onto the stack (correct starting point)
  S.push_back(P[kPrime + 0]);

  // Iterate from k'-1 to 0 (inclusive) in descending order
  for (int i = kPrime-1; i >= 0; --i) {
    while (S.size() > 1 && nonRightTurn(nextToTop(S), top(S), P[i])) {
      S.pop_back();
    }
    S.push_back(P[i]);
  }
  return S;
}


std::vector<double> algorithm4(std::vector<Point>& P, const std::vector<Point>& S, int kPrime) {
  std::vector<Point> Sprime(S.rbegin(), S.rend()); // Reverse stack
  std::vector<double> F0(kPrime + 1);

  // Iterate from k' to 1 (inclusive) in descending order
  for (int i = kPrime; i >= 1; --i) {
    F0[i] = slope(top(Sprime), nextToTop(Sprime));
    P[i].first = P[i - 1].first + P[i+1].first - P[i].first;
    P[i].second = P[i - 1].second + P[i+1].second - P[i].second;

    if (notBelow(P[i], top(Sprime), nextToTop(Sprime))) {
      continue;
    }

    Sprime.pop_back();
    while (Sprime.size() > 1 && nonRightTurn(P[i], top(Sprime), nextToTop(Sprime))) {
      Sprime.pop_back();
    }
    Sprime.push_back(P[i]);
  }
  return F0;
}

# prepareData

In [14]:
%%cpp -d
std::tuple<std::vector<double>, std::vector<double>, std::vector<double>> 
prepareData(const std::vector<Point>& calibrPoints) {
  // Sort and extract x and y coordinates
  std::vector<Point> ptsSorted(calibrPoints);
  std::sort(ptsSorted.begin(), ptsSorted.end());

  std::vector<double> xs(ptsSorted.size());
  std::vector<double> ys(ptsSorted.size());
  for (int i = 0; i < ptsSorted.size(); ++i) {
    xs[i] = ptsSorted[i].first;
    ys[i] = ptsSorted[i].second;
  }

  // Find unique points and calculate weights and cumulative sums
  std::vector<double> ptsUnique;
  std::vector<int> ptsInverse;
  std::vector<int> w;

  std::tie(ptsUnique,w,ptsInverse) = unique_with_counts_and_inverse(xs);

  std::vector<double> a(ptsUnique.size());
  for (size_t i = 0; i < ptsInverse.size(); ++i) {
    a[ptsInverse[i]] += ys[i];
  }

  std::vector<double> yCsd(a.size());
  std::partial_sum(a.begin(), a.end(), yCsd.begin()); // Cumulative sum of yCsd

  std::vector<double> xPrime(ptsUnique.size());
  std::partial_sum(w.begin(), w.end(), xPrime.begin()); // Cumulative sum of w

  return std::make_tuple(std::move(yCsd), std::move(xPrime), std::move(ptsUnique));
}


# getFVal

In [15]:
%%cpp -d

std::pair< std::vector<double>, std::vector<double> > 
getFVal(const std::vector<double>& F0, const std::vector<double>& F1,
                                 const std::vector<double>& ptsUnique, const std::vector<double>& testObjects) {
  std::vector<size_t> pos0(testObjects.size());
  std::vector<size_t> pos1(testObjects.size());

  // Use std::lower_bound for left-side search
  std::transform(testObjects.begin(), testObjects.end(), pos0.begin(),
                 [&ptsUnique](double testObject) {
                   return std::lower_bound(ptsUnique.begin(), ptsUnique.end(), testObject) - ptsUnique.begin();
                 });

  // Use std::upper_bound for right-side search, adjusting index for excluding the last element
  std::transform(testObjects.begin(), testObjects.end(), pos1.begin(),
                 [&ptsUnique](double testObject) {
                   return std::upper_bound(ptsUnique.begin(), ptsUnique.end() - 1, testObject) - ptsUnique.begin()+1;
                 });

  // Extract and return values from F0 and F1 based on positions
  std::vector<double> F0_vals(pos0.size());
  std::vector<double> F1_vals(pos0.size());
  for (size_t i = 0; i < pos0.size(); ++i) {
    F0_vals[i] = F0[pos0[i]];
    F1_vals[i] = F1[pos1[i]];
  }

  return std::make_pair(F0_vals, F1_vals);
}



# computeF

In [16]:
%%cpp -d
std::pair<std::vector<double>, std::vector<double>> 
computeF(const std::vector<double>& xPrime, const std::vector<double>& yCsd) {

  auto kPrime = xPrime.size();
  
  // Create P vector with Point values
  std::vector<Point> P(kPrime+2);

  P[-1+1] = Point(-1.0, -1.0); // Dummy point for index -1
  P[0+1] = Point(0.0, 0.0);
  for (int i = 0; i < kPrime; ++i) {
    P[i + 2] = Point(xPrime[i], yCsd[i]);
  }

  // Compute F1 using algorithms 1 and 2
  std::vector<Point> S = algorithm1(P, kPrime);
  std::vector<double> F1 = algorithm2(P, S, kPrime);

  // Update P with dummy point and modified P[kPrime + 1]
  P[0] = Point(0.0, 0.0);
  for (int i = 0; i < kPrime; ++i) {
    P[i + 1] = Point(xPrime[i], yCsd[i]);
  }
  // P[kPrime + 1] = P[kPrime] + Point(1.0, 0.0); // Adjust based on paper's correction
  P[kPrime + 1].first = P[kPrime].first + 1.0;
  P[kPrime + 1].second = P[kPrime].second;

  // Compute F0 using algorithms 3 and 4
  S = algorithm3(P, kPrime);
  std::vector<double> F0 = algorithm4(P, S, kPrime);

  return std::make_pair(F0, F1);
}

# ScoresToMultiProbs

In [17]:
%%cpp -d
std::pair<std::vector<double>, std::vector<double>> 
ScoresToMultiProbs(const std::vector<Point>& calibrPoints, const std::vector<double>& testObjects) {

  std::vector<double> yCsd, xPrime, ptsUnique;
  std::tie(yCsd,xPrime,ptsUnique) = prepareData(calibrPoints);

  std::vector<double> F0, F1;
  std::tie(F0,F1) = computeF(xPrime,yCsd);
    
  std::vector<double> p0, p1;
  std::tie(p0, p1) = getFVal(F0,F1,ptsUnique,testObjects);
  return std::make_pair(p0, p1);
}

In [18]:
calibrPoints = [(2.0,0.0),(3.0,1.0),(4.0,0.0),(5.0,0.0),(7.0,1.0),(8.0,1.0)]
testObjects = [1.0, 2.5, 7.2]

In [19]:
yPrime,yCsd,xPrime,ptsUnique = prepareData(calibrPoints)

In [20]:
res = ROOT.prepareData(calibrPoints)

In [21]:
yCsd_c,xPrime_c,ptsUnique_c = res
yCsd_c,xPrime_c,ptsUnique_c

(vector<double>{ 0.0000000, 1.0000000, 1.0000000, 1.0000000, 2.0000000, 3.0000000 },
 vector<double>{ 1.0000000, 2.0000000, 3.0000000, 4.0000000, 5.0000000, 6.0000000 },
 vector<double>{ 2.0000000, 3.0000000, 4.0000000, 5.0000000, 7.0000000, 8.0000000 })

In [22]:
ScoresToMultiProbs(calibrPoints, testObjects)

[-1 -1] [0 0] [1. 0.]
[-1 -1] [1. 0.] [2. 1.]
[1. 0.] [2. 1.] [3. 1.]
[-1 -1] [3. 1.] [4. 1.]
[-1 -1] [4. 1.] [5. 2.]
[4. 1.] [5. 2.] [6. 3.]
Pop!
Push!
Pop!
Push!
Push!
Pop!
Push!
Pop!
Push!
Push!
Algo 1 S: [array([7., 3.]), array([4., 1.]), array([1., 0.]), array([0, 0])]
F0:  [ 0.         -0.          0.25        0.25        0.25        0.5
  0.66666667]


(array([ 0. , -0. ,  0.5]), array([0.4, 0.5, 1. ]))

In [23]:
p0,p1 = ROOT.ScoresToMultiProbs(calibrPoints, testObjects)

In [24]:
p0, p1

(vector<double>{ 0.0000000, -0.0000000, 0.50000000 },
 vector<double>{ 0.40000000, 0.50000000, 1.0000000 })